In [3]:
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Загрузка токенизатора и модели BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Пример текста для токенизации
sentence = "molecule"

# Токенизация и создание входных тензоров
inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True, add_special_tokens=True)
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

# Генерация эмбеддингов с помощью модели BERT
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    last_hidden_states = outputs.last_hidden_state

# Усреднение скрытых состояний для получения эмбеддинга предложения
sentence_embedding = last_hidden_states.mean(dim=1)
print("Shape of Sentence Embedding:", sentence_embedding.shape)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Shape of Sentence Embedding: torch.Size([1, 768])


In [4]:
# Пример предложения для сравнения
example_sentence = "Pushkin"

# Токенизация и создание входных тензоров для примерного предложения
example_encoding = tokenizer.batch_encode_plus(
    [example_sentence],
    padding=True,
    truncation=True,
    return_tensors='pt',
    add_special_tokens=True
)
example_input_ids = example_encoding['input_ids']
example_attention_mask = example_encoding['attention_mask']

# Генерация эмбеддингов для примерного предложения
with torch.no_grad():
    example_outputs = model(example_input_ids, attention_mask=example_attention_mask)
    example_sentence_embedding = example_outputs.last_hidden_state.mean(dim=1)

# Вычисление косинусного сходства между исходным и примерным эмбеддингами предложений
similarity_score = cosine_similarity(sentence_embedding, example_sentence_embedding)

# Вывод результата
print("Cosine Similarity Score:", similarity_score[0][0])

Cosine Similarity Score: 0.50752294


In [11]:
from sklearn.cluster import KMeans

# Пример текстов для кластеризации
sentences = ["Tolstoy", "Russia", "Pushkin", "physics", "USA", "chemistry"]

# Генерация эмбеддингов для всех текстов
embeddings = []
for sentence in sentences:
    inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True, add_special_tokens=True)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        last_hidden_states = outputs.last_hidden_state
        sentence_embedding = last_hidden_states.mean(dim=1)
        embeddings.append(sentence_embedding.numpy())

embeddings = np.vstack(embeddings)

# Кластеризация текстов с помощью K-means
kmeans = KMeans(n_clusters=3)
kmeans.fit(embeddings)
labels = kmeans.labels_

# Вывод результатов кластеризации
for i, sentence in enumerate(sentences):
    print(f"Sentence: '{sentence}' is in cluster {labels[i]}")

Sentence: 'Tolstoy' is in cluster 1
Sentence: 'Russia' is in cluster 2
Sentence: 'Pushkin' is in cluster 1
Sentence: 'physics' is in cluster 0
Sentence: 'USA' is in cluster 2
Sentence: 'chemistry' is in cluster 0


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
